<a href="https://colab.research.google.com/github/PokeMaster16/Machine-Learning-Projects/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
#import tensorflow as tf
import pandas as pd
#from tensorflow import keras
!pip install tensorflow-datasets
#import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

#print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# --- Load and preprocess ---
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=["label", "message"])

# Convert string labels to integers
label_map = {'ham': 0, 'spam': 1}
train_df['label'] = train_df['label'].map(label_map)
test_df['label'] = test_df['label'].map(label_map)

# --- Features and labels ---
X_train = train_df['message']
y_train = train_df['label']
X_test = test_df['message']
y_test = test_df['label']

# --- TF-IDF ---
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# --- Train model ---
model = LogisticRegression(class_weight='balanced')
model.fit(X_train_tfidf, y_train)

# --- Classifier function ---
def classify_message(message):
    msg_tfidf = vectorizer.transform([message])
    proba = model.predict_proba(msg_tfidf)[0]
    prediction = int(model.predict(msg_tfidf)[0])  # Now it's an integer (0 or 1)
    label = "Spam" if prediction == 1 else "Ham"
    confidence = proba[prediction]
    return f"{label} (Confidence: {confidence:.2f})"

# --- Try it ---
print(classify_message("Congratulations! You've won a free ticket. Call now!"))
print(classify_message("Hi, can we meet at 5 pm?"))


In [ ]:
def classify_message(message):
    msg_tfidf = vectorizer.transform([message])
    proba = model.predict_proba(msg_tfidf)[0]
    prediction = int(model.predict(msg_tfidf)[0])  # Make sure it's an integer
    label = "Spam" if prediction == 1 else "Ham"
    confidence = proba[prediction]  # Now safe
    return f"{label} (Confidence: {confidence:.2f})"

# Test
print(classify_message("Congratulations! You've won a free ticket. Call now!"))
print(classify_message("Hi, can we meet at 5 pm?"))


In [ ]:
def predict_message(pred_text):
    # Transform the message
    msg_tfidf = vectorizer.transform([pred_text])

    # Get probability of being spam (class 1)
    spam_prob = model.predict_proba(msg_tfidf)[0][1]  # Index 1 = probability of class 'spam'

    # Choose label based on threshold
    label = 'spam' if spam_prob >= 0.5 else 'ham'

    # Return result in required format
    return [spam_prob, label]

# Example usage
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
